In [3]:
import numpy as np
import pandas as pd
import random as rd
from numpy.random import random
from scipy import interpolate
import matplotlib.pyplot as plt
import scipy.interpolate as sp_interp
import cProfile
from scipy import optimize

In [39]:
class simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.generate_arrival()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from server 1
        self.t_departure2=float('inf')      #departure time from server 2
        self.dep_sum1=0                     #Sum of service times by server 1
        self.dep_sum2=0                     #Sum of service times by server 2
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                       #current number in queue
        self.num_in_queue=0              #customersin queue
        self.num_of_departures1=0           ##customers served by server 1
        self.num_of_departures2=0           ##customers served by server 2
        self.num_in_system=0  
    def time_adv(self):                                                       
        t_event=min(self.t_arrival,self.t_departure1,self.t_departure2)  
        self.total_wait_time += (self.num_in_q*(t_event-self.clock))
        self.clock=t_event
                
        if self.t_arrival<self.t_departure1 and self.t_arrival<self.t_departure2:
            self.arrival()
        elif self.t_departure1<self.t_arrival and self.t_departure1<self.t_departure2:
            self.server1()
        else:
            self.server2()
    
            
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.num_in_q == 0:                                 #schedule next departure or arrival depending on state of servers
            if self.state_T1==1 and self.state_T2==1:
                self.num_in_q+=1
                self.num_in_queue+=1
                self.t_arrival=self.clock+self.generate_arrival()    
            elif self.state_T1==0 and self.state_T2==0:  #if both server are avialible
                if np.random.choice([0,1])==1:   #random binary choice 0,1 
                    self.state_T1=1
                    self.dep1= self.server1_time()
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.t_arrival=self.clock+self.generate_arrival()   
                else:
                    self.state_T2=1
                    self.dep2= self.server2_time()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival=self.clock+self.generate_arrival()                      

            elif self.state_T1==0 and self.state_T2 ==1:       ## server1 is2 empty sever 2 busy
                self.dep1= self.server1_time()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.generate_arrival()
                self.state_T1=1
            else:
                self.dep1= self.server2_time()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.generate_arrival()
                self.state_T2=1
            
        else:
            self.num_in_q+=1
            self.num_in_queue+=1 
            self.t_arrival=self.clock + self.generate_arrival()
            
            
            
    def server1(self): # of departure 
        self.num_of_departures1 += 1
        if self.num_in_q>0:
            self.dep1 = self.server1_time()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock+self.dep1
            self.num_in_q -=1
        else:
            self.t_departure1 = float('inf')
            self.state_T1 = 0
    
    def server2(self): # of departure 
        self.num_of_departures2 += 1
        if self.num_in_q>0:
            self.dep1 = self.server2_time()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock+self.dep1
            self.num_in_q -=1
        else:
            self.t_departure2 = float('inf')
            self.state_T2 = 0            
    
        
    def generate_arrival(self):    
        return np.random.poisson(6)
    def server1_time(self):
        return np.random.exponential(4)
    def server2_time(self):
        return np.random.exponential(3)        

In [40]:
s=simulation()
df=pd.DataFrame(columns=['Average interarrival time','Average service time teller1','Average service time teller 2','Utilization teller 1','Utilization teller 2','People who had to wait in line','Total average wait time'])

for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv()
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.num_in_queue,s.total_wait_time],index=df.columns)
    df=df.append(a,ignore_index=True)  

C:\Users\Freem\AppData\Local\Temp\ipykernel_19136\1791326018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(a,ignore_index=True)


ZeroDivisionError: division by zero

In [41]:
import numpy as np
import pandas as pd

class Bank_Simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from s1
        self.t_departure2=float('inf')      #departure time from se2
        self.dep_sum1=0                     #Sum of service times by s 1
        self.dep_sum2=0                     #Sum of service times by s 2
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_in_system=0                #current number of customers in system
        self.num_of_departures1=0           #number of customers served by s1  
        self.num_of_departures2=0           #number of customers served by s 2 

        
    def time_adv(self):                                                       #timing routine
        t_next_event=min(self.t_arrival,self.t_departure1,self.t_departure2)  #determine time of next event
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure1 and self.t_arrival<self.t_departure2:
            self.arrival()
        elif self.t_departure1<self.t_arrival and self.t_departure1<self.t_departure2:
            self.s1()
        else:
            self.s2()
            
        
    
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.num_in_q == 0:                              #schedule next departure or arrival depending on state of servers
            if self.state_T1==1 and self.state_T2==1:
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival=self.clock+self.gen_int_arr()
                
                
            elif self.state_T1==0 and self.state_T2==0:
                
                if np.random.choice([0,1])==1:
                    self.state_T1=1
                    self.dep1= self.server1_time()
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.t_arrival=self.clock+self.gen_int_arr()

                else:
                    self.state_T2=1
                    self.dep2= self.server1_time2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival=self.clock+self.gen_int_arr()

                    
            elif self.state_T1==0 and self.state_T2 ==1:    #if server 2 is busy customer goes to server 1
                self.dep1= self.server1_time()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T1=1
            else:                                           #otherwise customer goes to server 2
                self.dep2= self.server1_time2()
                self.dep_sum2 += self.dep2
                self.t_departure2=self.clock + self.dep2
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T2=1
        
        else :
            self.num_in_q+=1
            self.number_in_queue+=1                             #if queue length is less than 4 generate next arrival and make customer join queue
            self.t_arrival=self.clock + self.gen_int_arr()
            

                
            

    
    def s1(self):              #departure from server 2
        self.num_of_departures1 += 1
        self.num_in_system -= 1 
        if self.num_in_q>0:
            self.dep1= self.server1_time()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  
    
    def s2(self):              #departure from server 1
        self.num_of_departures2 += 1
        self.num_in_system -= 1
        if self.num_in_q>0:
            self.dep2= self.server1_time2()
            self.dep_sum2 += self.dep2
            self.t_departure2=self.clock + self.dep2
            self.num_in_q-=1
        else:
            self.t_departure2=float('inf')
            self.state_T2=0
            
    
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return np.random.poisson(6)
    
    def server1_time(self):                                #function to generate service time for s 1 using inverse trnasform
        return  np.random.exponential(4)
    
    def server1_time2(self):                                #function to generate service time for s 2 using inverse trnasform
        return  np.random.exponential(3)


   
s=Bank_Simulation()
df=pd.DataFrame(columns=['Average interarrival time','Average service time teller1','Average service time teller 2','Utilization teller 1','Utilization teller 2','People who had to wait in line','Total average wait time'])


for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.number_in_queue,s.total_wait_time],index=df.columns)
    df=df.append(a,ignore_index=True)   


C:\Users\Freem\AppData\Local\Temp\ipykernel_19136\809874520.py:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(a,ignore_index=True)


In [42]:
for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.number_in_queue,s.total_wait_time],index=df.columns)
    df=df.append(a,ignore_index=True)   

C:\Users\Freem\AppData\Local\Temp\ipykernel_19136\2988234934.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(a,ignore_index=True)


In [45]:
df

,Average interarrival time,Average service time teller1,Average service time teller 2,Utilization teller 1,Utilization teller 2,People who had to wait in line,Total average wait time
0,6.125000,4.339743,2.968568,0.371978,0.205982,1.0,0.164579
1,5.176271,4.517862,3.349278,0.427116,0.316639,1.0,0.919377
2,6.179487,5.095578,1.846427,0.401726,0.137907,0.0,0.000000
3,6.579468,4.946114,3.301180,0.365716,0.257650,1.0,4.537510
4,5.500000,4.723824,2.812199,0.448959,0.220793,1.0,6.897705
...,...,...,...,...,...,...,...
195,6.368421,3.645069,4.264894,0.346432,0.229106,2.0,3.563567
196,6.581948,3.647844,2.187842,0.239663,0.188660,0.0,0.000000
197,6.205128,4.361804,3.635057,0.342456,0.270376,0.0,0.000000
198,5.878049,3.435658,3.531998,0.313629,0.263801,4.0,4.637420
